In [1]:
import glob
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import os
import gensim
from annoy import AnnoyIndex

type_labels = {'clicks':0, 'carts':1, 'orders':2}
predTypes = ['orders']

def load_df(path):    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(path)):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

def load_ValA(SET):
    if SET == 1:
        return load_df('../input/split_2-1_pqt/test_parquets/*')
    elif SET == 2:
        return pd.read_parquet('../input/splited/test.parquet')
    elif SET == 3:
        return load_df('../input/parquets/test_parquets/*')

In [2]:
toConcat = []
for SET in [2]:
    val_A = load_ValA(SET)
    val_A = val_A.sort_values(['session', 'ts'], ascending=[True, False]).reset_index(drop=True)
    val_A = val_A.drop_duplicates((['session', 'aid'])).reset_index(drop=True)

    val_A['n'] = val_A.groupby('session').aid.cumcount()
    val_A = val_A.loc[val_A.n < 3].drop('n',axis=1)
    
    for predType in predTypes:
        for sub in range(6):
            print(SET, predType, sub)
            candidate = pd.read_parquet(f'../output/newSplited/candidates/set{SET}_top_100/suggester_addLast/{predType}_{sub}.pqt')[['session', 'aid']]
            candidate['session'] = candidate.session.astype('int32')
            df =val_A.merge(candidate, on='session')
            df = df[['aid_x', 'aid_y']].drop_duplicates()
            toConcat.append(df)

aidxaid = pd.concat(toConcat, axis=0, ignore_index=True)
aidxaid = aidxaid.drop_duplicates().reset_index(drop=True)
aidxaid

2 orders 0
2 orders 1
2 orders 2
2 orders 3
2 orders 4
2 orders 5


,aid_x,aid_y
0,11830,11830
1,11830,1732105
2,11830,588923
3,11830,884502
4,11830,876129
...,...,...
125448412,561560,576949
125448413,561560,1534690
125448414,561560,122983
125448415,561560,1257293


In [3]:
aid2vec_model = gensim.models.KeyedVectors.load_word2vec_format(f'../output/newSplited/savedModel/set_{SET}/otto_aid2vec_5d.bin', binary=True)

# aid2idx = {int(aid): i for i, aid in enumerate(aid2vec_model.index_to_key)}
# index = AnnoyIndex(8, 'euclidean')
# for _, idx in aid2idx.items():
#     index.add_item(idx, aid2vec_model.vectors[idx])
# index.build(10)

Merge with old

In [ ]:
aidxaid_old = pd.read_parquet('../output/newSplited/features/suggester_addLast/aid2aid_annoySim.pqt')
aidxaid_old

In [ ]:
aidxaid = aidxaid.merge(aidxaid_old, on=['aid_x', 'aid_y'], how='left').fillna(-1)
aidxaid[['aid_x', 'aid_y']] = aidxaid[['aid_x', 'aid_y']].astype('int32')

In [6]:
aidxaid.loc[aidxaid.annoySim == -1, 'annoySim'] = aidxaid[aidxaid.annoySim == -1].progress_apply(lambda x: aid2vec_model.similarity(str(int(x.aid_x)), str(int(x.aid_y))), axis=1)
! mkdir ../output/newSplited/features/suggester_addLast/set_{SET}
aidxaid.to_parquet(f'../output/newSplited/features/suggester_addLast/set_{SET}/aid2aid_annoySim.pqt')


A Jupyter Widget

Create new

In [4]:
chunkSize = 30_000_000
i = 0
n = len(aidxaid)
while (i*chunkSize < n):
    if i == 0:
        annoySim = aidxaid[i*chunkSize : (i+1)*chunkSize].progress_apply(lambda x: aid2vec_model.similarity(str(x.aid_x), str(x.aid_y)), axis=1)
    else:
        annoySim = pd.concat([annoySim, aidxaid[i*chunkSize : (i+1)*chunkSize].progress_apply(lambda x: aid2vec_model.similarity(str(x.aid_x), str(x.aid_y)), axis=1)], axis=0, ignore_index=True)
    # annoySim.append(aidxaid[i*chunkSize : (i+1)*chunkSize].progress_apply(lambda x: index.get_distance(aid2idx[x.aid_x], aid2idx[x.aid_y]), axis=1))
    i += 1
aidxaid['annoySim'] = annoySim
! mkdir ../output/newSplited/features/suggester_addLast/set_{SET}
aidxaid.to_parquet(f'../output/newSplited/features/suggester_addLast/set_{SET}/aid2aid_annoySim.pqt')

# test = aidxaid[:1000]
# test['annoySim'] = test.progress_apply(lambda x: index.get_distance(aid2idx[x.aid_x], aid2idx[x.aid_y]), axis=1)
# test

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget